In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


--2022-12-05 23:20:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.44MB/s    in 0.2s    

2022-12-05 23:20:15 (5.44 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Read in 3 Crime Datasets

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
bo_url = "https://capstone-crime-bucket.s3.amazonaws.com/crime.csv"
spark.sparkContext.addFile(bo_url)
bo_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("crime.csv"), sep=",", header=True, inferSchema=True)
bo_df.show



<bound method DataFrame.show of DataFrame[INCIDENT_NUMBER: string, OFFENSE_CODE: int, OFFENSE_CODE_GROUP: string, OFFENSE_DESCRIPTION: string, DISTRICT: string, REPORTING_AREA: string, SHOOTING: string, OCCURRED_ON_DATE: timestamp, YEAR: int, MONTH: int, DAY_OF_WEEK: string, HOUR: int, UCR_PART: string, STREET: string, Lat: double, Long: double, Location: string]>

In [7]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

la_url = "https://capstone-crime-bucket.s3.amazonaws.com/Crimes_2012-2016.csv"
spark.sparkContext.addFile(la_url)
la_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("Crimes_2012-2016.csv"), sep=",", header=True, inferSchema=True)
la_df.show

<bound method DataFrame.show of DataFrame[_c0: int, DR.NO: int, DATE.OCC: string, TIME.OCC: int, AREA: int, AREA.NAME: string, RD: int, Crm.Cd: int, CrmCd.Desc: string, Location.1: string, lat: double, long: double, Year: int]>

In [8]:
# Read in data from S3 Buckets
from pyspark import SparkFiles


bal_url = "https://capstone-crime-bucket.s3.amazonaws.com/Baltimore_Crime_Data.csv"
spark.sparkContext.addFile(bal_url)
bal_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("Baltimore_Crime_Data.csv"), sep=",", header=True, inferSchema=True)
bal_df.show



<bound method DataFrame.show of DataFrame[CrimeDate: string, CrimeTime: string, CrimeCode: string, Location: string, Description: string, Inside/Outside: string, Weapon: string, Post: int, District: string, Neighborhood: string, Longitude: double, Latitude: double, Location 1: string, Premise: string, Total Incidents: int]>

# DataFrames